In [8]:
from biodivine_aeon import *
from pathlib import Path

In [9]:
model = BooleanNetwork.from_file("model_map1.aeon")
print(model)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=37, implicit_parameters=22)


In [10]:
stg = AsynchronousGraph(model)

First, we identify the unknown functions that can be safely eliminated because they have only one interpretations that satisfies all the requirements regarding their input monotonicity and essentiality.

In [11]:
to_eliminate = []
for param in model.explicit_parameters():
    print(model.get_explicit_parameter_name(param))
    count = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    if count == 1:
        to_eliminate.append(param)

cat_MKK7_MAP2K7__phosphorylated_67
 >  ColorModel({'cat_MKK7_MAP2K7__phosphorylated_67': 'x_0'})
cat_Cbp_PAG_phosphorylated_palmytoylated_69
 >  ColorModel({'cat_Cbp_PAG_phosphorylated_palmytoylated_69': 'x_0'})
cat_Btk_phosphorylated_Cytosol_42
 >  ColorModel({'cat_Btk_phosphorylated_Cytosol_42': 'x_0'})
cat_Fos_phosphorylated_68
 >  ColorModel({'cat_Fos_phosphorylated_68': 'x_0'})
cat_MEK2_MAP2K2__phosphorylated_41
 >  ColorModel({'cat_MEK2_MAP2K2__phosphorylated_41': 'x_0'})
cat_SOS_Grb2_complex_5
 >  ColorModel({'cat_SOS_Grb2_complex_5': 'x_0'})
 >  ColorModel({'cat_SOS_Grb2_complex_5': 'true'})
cat_Erk_MAPK1_3__phosphorylated_49
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '(x_0 & x_1)'})
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '((!x_0 & x_1) | x_0)'})
cat_PLCG1_phosphorylated_64
 >  ColorModel({'cat_PLCG1_phosphorylated_64': 'x_0'})
cat_Shc_phosphorylated_56
 >  ColorModel({'cat_Shc_phosphorylated_56': '(x_0 & x_1)'})
 >  ColorModel({'cat_Shc_phosphorylated

In [12]:
print(to_eliminate)

[ParameterId(27), ParameterId(6), ParameterId(0), ParameterId(14), ParameterId(24), ParameterId(31), ParameterId(10), ParameterId(15), ParameterId(17), ParameterId(3), ParameterId(13), ParameterId(33), ParameterId(2), ParameterId(4), ParameterId(21), ParameterId(1), ParameterId(25), ParameterId(30), ParameterId(23), ParameterId(36), ParameterId(28), ParameterId(32), ParameterId(19), ParameterId(16), ParameterId(12), ParameterId(26)]


In [13]:
m = next(stg.mk_unit_colors().items(retained = to_eliminate))

In [7]:
model_simplified = m.instantiate(model)

PanicException: assertion failed: translation_map.contains_key(&var) || param_map.contains_key(&var)

In [13]:
Path("model_map1.simplified.aeon").write_text(model_simplified.to_aeon())

NameError: name 'model_simplified' is not defined

In [9]:
stg = AsynchronousGraph(model_simplified)

In [10]:
# This identifies model outputs. In the next step, we restrict them manually to
# a subset of outputs that are biologically interpretable.
outputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.successors(var)) == 0:
        outputs.append(var)
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Fyn_phosphorylated_phosphorylated_palmytoylated',
 'I_kB_phosphorylated',
 'Jun_Fos_complex',
 'LAT2_Grb2_Shc_SOS_H_RAS_RAF1_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex',
 'c_JUN_phosphorylated_nucleus']

In [11]:
# Actually relevant outputs:
outputs = ['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']

In [12]:
inputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.predecessors(var)) == 0:
        inputs.append(var)
inputs

['Akt',
 'Bcr',
 'Btk',
 'CRAC',
 'Csk',
 'Dok1_RasGAP_complex',
 'Fyn_phosphorylated_palmytoylated',
 'GRAP2',
 'H_RAS_Plasma_space_Membrane',
 'LAT2_Grb2_complex',
 'PAK2',
 'PIP2_simple_molecule',
 'PI_3K',
 'PKC_space_Theta',
 'PLCG1',
 'RAF1_Cytosol',
 'RASGRP1',
 'Rac1',
 'Shc_Grb2_complex',
 'Syk',
 'Vav1',
 'cCbl']

In [13]:
params = []
for p in model_simplified.explicit_parameter_names():
    params.append((p, model_simplified.get_explicit_parameter_arity(p)))
params

[('cat_Calmodulin_Calcineurin_complex_19', 2),
 ('cat_DAG_simple_molecule_65', 2),
 ('cat_Elk1_phosphorylated_53', 3),
 ('cat_Erk_MAPK1_3__phosphorylated_49', 2),
 ('cat_FceRI_Syk_complex_Cytosol_1_24', 1),
 ('cat_IP3_simple_molecule_57', 2),
 ('cat_JNK_phosphorylated_70', 2),
 ('cat_LAT2_phosphorylated_palmytoylated_47', 2),
 ('cat_PIP3_Akt_complex_8', 1),
 ('cat_SOS_Grb2_complex_5', 1),
 ('cat_Shc_phosphorylated_56', 2)]

In [14]:
ctx = SymbolicSpaceContext(model_simplified)
stg = AsynchronousGraph(model_simplified, ctx)

Compute minimal trap spaces across all interpretations. These should almost exactly correspond to model attractors (this can take a few minutes).

In [15]:
traps = TrapSpaces.essential_symbolic(ctx, stg, ctx.mk_unit_colored_spaces(stg))

Start symbolic essential trap space search with 2024261176623274600000000000000000000000000000000000000[nodes:236] candidates.
 > Merge. New result has 123457 BDD nodes. Remaining constraints: 10.
 > Merge. New result has 152944 BDD nodes. Remaining constraints: 9.
 > Merge. New result has 158220 BDD nodes. Remaining constraints: 8.
 > Merge. New result has 218609 BDD nodes. Remaining constraints: 7.
 > Merge. New result has 195497 BDD nodes. Remaining constraints: 6.
 > Merge. New result has 209627 BDD nodes. Remaining constraints: 5.
 > Merge. New result has 257497 BDD nodes. Remaining constraints: 4.
 > Merge. New result has 248221 BDD nodes. Remaining constraints: 3.
 > Merge. New result has 314759 BDD nodes. Remaining constraints: 2.
 > Merge. New result has 429900 BDD nodes. Remaining constraints: 1.
 > Merge. New result has 884192 BDD nodes. Remaining constraints: 0.
Merge finished with 884192 BDD nodes.
Found 77309411328x71795760[nodes:884192] essential trap spaces.


In [16]:
min_traps = TrapSpaces.minimize(ctx, traps)

Start minimal subspace search with 77309411328x71795760[nodes:884192] candidates.
Computing super-spaces: 382747566778465550000000000000000000000[nodes:580482].
Computing super-spaces: 765425037549340900000000000000000000000[nodes:580480].
Computing super-spaces: 1530850075098681800000000000000000000000[nodes:580476].
Computing super-spaces: 3061700150197363500000000000000000000000[nodes:580473].
Computing super-spaces: 6123400300394727000000000000000000000000[nodes:580467].
Computing super-spaces: 12246800600789454000000000000000000000000[nodes:580462].
Computing super-spaces: 24493601201578910000000000000000000000000[nodes:580446].
Computing super-spaces: 48958042464000285000000000000000000000000[nodes:580419].
Computing super-spaces: 97886924988843040000000000000000000000000[nodes:580369].
Computing super-spaces: 195773849977686080000000000000000000000000[nodes:580303].
Computing super-spaces: 391547699955372150000000000000000000000000[nodes:580237].
Computing super-spaces: 78309539

In [17]:
min_traps

99821488600000000000000000000000000[nodes:580117].
Computing super-spaces: 3132381599642977000000000000000000000000000[nodes:579659].
Computing super-spaces: 6264763199285954000000000000000000000000000[nodes:579675].
Computing super-spaces: 12529526398571910000000000000000000000000000[nodes:579177].
Computing super-spaces: 25059052797143820000000000000000000000000000[nodes:578761].
Computing super-spaces: 50117172476234590000000000000000000000000000[nodes:577837].
Computing super-spaces: 100234344952469190000000000000000000000000000[nodes:577197].
Computing super-spaces: 200468689904938380000000000000000000000000000[nodes:576249].
Computing super-spaces: 400937379809876760000000000000000000000000000[nodes:573285].
Computing super-spaces: 801874759619753500000000000000000000000000000[nodes:569983].
Computing super-spaces: 1603749519239507000000000000000000000000000000[nodes:567519].
Computing super-spaces: 3207499038479014000000000000000000000000000000[nodes:566495].
Computing super-spa

ColoredSpaceSet(cardinality=228493099008, colors=77309411328, spaces=35929632, symbolic_size=588576)

ces: 6414938357402633000000000000000000000000000000[nodes:562971].
Computing super-spaces: 12829816995249872000000000000000000000000000000[nodes:559447].
Computing super-spaces: 25659633990499745000000000000000000000000000000[nodes:552671].
Computing super-spaces: 51319267980999490000000000000000000000000000000[nodes:545895].
Computing super-spaces: 102637488572873600000000000000000000000000000000[nodes:539335].
Computing super-spaces: 205273929756621800000000000000000000000000000000[nodes:532775].
Computing super-spaces: 410546812124118200000000000000000000000000000000[nodes:526215].
Computing super-spaces: 820992027503074400000000000000000000000000000000[nodes:520087].
Computing super-spaces: 1641882458260986600000000000000000000000000000000[nodes:513959].
Computing super-spaces: 3283764916521973000000000000000000000000000000000[nodes:507183].
Computing super-spaces: 6567529833043946000000000000000000000000000000000[nodes:502527].
Computing super-spaces: 13135059666087893000000000000

Compute the number of output value combinations that can actually appear in the model, and the distribution of output values within these combinations.

In [29]:
phenotype_iterator = min_traps.spaces().items(retained = outputs)
phenotype_count = sum(1 for _ in phenotype_iterator)
print(f"Phenotypes: {phenotype_count}")

Phenotypes: 84


In [30]:
is_zero = { var: 0 for var in outputs }
is_one = { var: 0 for var in outputs }
is_any = { var: 0 for var in outputs }
for phenotype in min_traps.spaces().items(retained = outputs):
    phenotype_traps = min_traps.intersect_spaces(phenotype.to_symbolic())
    for (var, value) in phenotype.items():        
        var = ctx.get_network_variable_name(var)
        if value == True:
            is_one[var] += 1
        elif value == False:
            is_zero[var] += 1
        else:
            is_any[var] += 1

print("Zero:", is_zero)
print("One:", is_one)
print("Any:", is_any)

Zero: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 52, 'Elk1_phosphorylated': 20, 'Jun_Fos_complex': 52, 'NFAT_nucleus': 26, 'NF_kappa_B_nucleus': 36, 'PIP3_Akt_complex': 42}
One: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 20, 'Elk1_phosphorylated': 24, 'Jun_Fos_complex': 12, 'NFAT_nucleus': 58, 'NF_kappa_B_nucleus': 30, 'PIP3_Akt_complex': 42}
Any: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 12, 'Elk1_phosphorylated': 40, 'Jun_Fos_complex': 20, 'NFAT_nucleus': 0, 'NF_kappa_B_nucleus': 18, 'PIP3_Akt_complex': 0}


In [31]:
def cardinality_without_inputs(ctx, inputs, set):
    # Project the BDD to remove inputs.
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    all_vars = ctx.bdd_variable_set().variable_count()
    function_vars = sum(len(x) for x in ctx.explicit_functions_bdd_variables().values())

    general_cardinality = set_bdd.cardinality()
    return int(general_cardinality / (2 ** (all_vars - function_vars)))

def prune_inputs(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    return ColorSet(ctx, set_bdd)

def prune_inputs_forall(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_for_all(ignore_vars)

    return ColorSet(ctx, set_bdd)

Build a table showing the multiplicity of various attractor combinations for each output variable:

In [32]:
print(f"0\t1\t*\t0-1\t1-*\t0-*\t0-1-*\tname")

for var in outputs:
    p_var = ctx.get_positive_space_variable(var)
    n_var = ctx.get_negative_space_variable(var)
    var_is_one = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: False }))
    var_is_zero = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: False, n_var: True }))
    var_is_any = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: True }))

    one_colors = min_traps.intersect_spaces(var_is_one).colors()
    zero_colors = min_traps.intersect_spaces(var_is_zero).colors()
    any_colors = min_traps.intersect_spaces(var_is_any).colors()

    # one_colors = prune_inputs(ctx, inputs, one_colors)
    # zero_colors = prune_inputs(ctx, inputs, zero_colors)
    # any_colors = prune_inputs(ctx, inputs, any_colors)

    one = one_colors.minus(zero_colors).minus(any_colors)
    zero = zero_colors.minus(one_colors).minus(any_colors)
    any = any_colors.minus(zero_colors).minus(one_colors)
    zero_one = one_colors.intersect(zero_colors).minus(any_colors)
    one_any = one_colors.intersect(any_colors).minus(zero_colors)
    zero_any = zero_colors.intersect(any_colors).minus(one_colors)
    zero_one_any = zero_colors.intersect(one_colors).intersect(any_colors)
    
    # c_one = cardinality_without_inputs(ctx, inputs, one)
    # c_zero = cardinality_without_inputs(ctx, inputs, zero)
    # c_any = cardinality_without_inputs(ctx, inputs, any)
    # c_zero_one = cardinality_without_inputs(ctx, inputs, zero_one)
    # c_one_any = cardinality_without_inputs(ctx, inputs, one_any)
    # c_zero_any = cardinality_without_inputs(ctx, inputs, zero_any)
    # c_zero_one_any = cardinality_without_inputs(ctx, inputs, zero_one_any)

    c_one = one.cardinality()
    c_zero = zero.cardinality()
    c_any = any.cardinality()
    c_zero_one = zero_one.cardinality()
    c_one_any = one_any.cardinality()
    c_zero_any = zero_any.cardinality()
    c_zero_one_any = zero_one_any.cardinality()

    print(f"{c_zero}\t{c_one}\t{c_any}\t{c_zero_one}\t{c_one_any}\t{c_zero_any}\t{c_zero_one_any}\t{var}")    

0	1	*	0-1	1-*	0-*	0-1-*	name
76880019456	207618048	7077888	207618048	0	7077888	0	Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex
76910952448	351272960	47185920	0	0	0	0	Elk1_phosphorylated
77224476672	56623104	28311552	0	0	0	0	Jun_Fos_complex
0	38654705664	0	38654705664	0	0	0	NFAT_nucleus
75591843840	1703411712	14155776	0	0	0	0	NF_kappa_B_nucleus
75723964416	1585446912	0	0	0	0	0	PIP3_Akt_complex


Finally, let us compare the partial network to the original CaSQ network:

In [33]:
model_known = BooleanNetwork.from_file("model_map1.full.aeon")
print(model_known)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=0, implicit_parameters=22)


In [34]:
ctx_known = SymbolicSpaceContext(model_known)
stg_known = AsynchronousGraph(model_known, ctx_known)

In [35]:
traps_known = TrapSpaces.essential_symbolic(ctx_known, stg_known, ctx_known.mk_unit_colored_spaces(stg_known))

Start symbolic essential trap space search with 109823197516453700000000000000000000000000000000000[nodes:184] candidates.
 > Merge. New result has 109532 BDD nodes. Remaining constraints: 3.
 > Merge. New result has 130676 BDD nodes. Remaining constraints: 2.
 > Merge. New result has 152921 BDD nodes. Remaining constraints: 1.
 > Merge. New result has 317507 BDD nodes. Remaining constraints: 0.
Merge finished with 317507 BDD nodes.
Found 4194304x24182784[nodes:317507] essential trap spaces.


In [36]:
min_traps_known = TrapSpaces.minimize(ctx_known, traps_known)

Start minimal subspace search with 4194304x24182784[nodes:317507] candidates.
Computing super-spaces: 20766652132938854000000000000000000[nodes:213005].
Computing super-spaces: 41530768964677250000000000000000000[nodes:213003].
Computing super-spaces: 83061537929354500000000000000000000[nodes:212999].
Computing super-spaces: 166123075858709000000000000000000000[nodes:212996].
Computing super-spaces: 332246151717418000000000000000000000[nodes:212990].
Computing super-spaces: 664492303434836000000000000000000000[nodes:212983].
Computing super-spaces: 1328984606869672000000000000000000000[nodes:212971].
Computing super-spaces: 2656914528439954200000000000000000000[nodes:212949].
Computing super-spaces: 5312774371580518600000000000000000000[nodes:212919].
Computing super-spaces: 10625548743161037000000000000000000000[nodes:212885].
Computing super-spaces: 21251097486322074000000000000000000000[nodes:212851].
Computing super-spaces: 42502194972644150000000000000000000000[nodes:212833].
Comp

In [37]:
min_traps_transferred = ColoredSpaceSet(ctx, ctx.transfer_from(min_traps_known.to_bdd(), ctx_known))

In [38]:
equivalent = prune_inputs_forall(ctx, inputs, min_traps.intersect(min_traps_transferred).colors())
print("All models:", cardinality_without_inputs(ctx, inputs, stg.mk_unit_colors()))
print("Equivalent models:", cardinality_without_inputs(ctx, inputs, equivalent))

All models: 18432
Equivalent models: 128


Let's examine which funtions are fixed/free in the 128 equivalent models:

In [39]:
for param in model_simplified.explicit_parameters():
    print(model_simplified.get_explicit_parameter_name(param))
    count = 0
    for m in equivalent.items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    total = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        total += 1
    print(f"{count}/{total}")

cat_PIP3_Akt_complex_8
 >  ColorModel({'cat_PIP3_Akt_complex_8': 'x_0'})
1/2
cat_LAT2_phosphorylated_palmytoylated_47
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '((!x_0 & x_1) | x_0)'})
1/2
cat_SOS_Grb2_complex_5
 >  ColorModel({'cat_SOS_Grb2_complex_5': 'x_0'})
1/2
cat_IP3_simple_molecule_57
 >  ColorModel({'cat_IP3_simple_molecule_57': '(x_0 & x_1)'})
 >  ColorModel({'cat_IP3_simple_molecule_57': '((!x_0 & x_1) | x_0)'})
2/2
cat_DAG_simple_molecule_65
 >  ColorModel({'cat_DAG_simple_molecule_65': '((!x_0 & x_1) | x_0)'})
1/2
cat_Calmodulin_Calcineurin_complex_19
 >  ColorModel({'cat_Calmodulin_Calcineurin_complex_19': '(x_0 & x_1)'})
 >  ColorModel({'cat_Calmodulin_Calcineurin_complex_19': 'x_0'})
 >  ColorModel({'cat_Calmodulin_Calcineurin_complex_19': 'x_1'})
 >  ColorModel({'cat_Calmodulin_Calcineurin_complex_19': '((!x_0 & x_1) | x_0)'})
4/4
cat_Erk_MAPK1_3__phosphorylated_49
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '(x_0 & x_1)'})
 >  ColorModel({'c